In [36]:
import metadrive
from metadrive.envs.base_env import BaseEnv
from metadrive.obs.observation_base import ObservationBase
from metadrive.constants import DEFAULT_AGENT
from metadrive.utils import Config
from metadrive.manager.scenario_map_manager import ScenarioMapManager
from metadrive.manager.base_manager import BaseManager
from metadrive.scenario.scenario_description import ScenarioDescription
from metadrive.component.vehicle_navigation_module.trajectory_navigation import TrajectoryNavigation



In [55]:
import os
import pickle

path = "~/myworkspace/scenarionet/waymo/waymo_0/sd_waymo_v1.2_7bdee109da62b185.pkl"
path = os.path.expanduser(path)

with open(path, "rb") as f:
    data = pickle.load(f)

In [56]:
class InMemoryScenarioDataManager(BaseManager):
    def __init__(self, scenario):
        super().__init__()
        self.current_scenario = scenario
    
    def get_scenario(self, i, should_copy=False):
        return self.current_scenario

class ReplayEnv(BaseEnv):
    @classmethod
    def default_config(cls) -> Config:
        config = super().default_config()
        config.update({
            "start_seed": 0,
            "num_scenarios": 1,
            "start_scenario_index": 0,
            "no_map": False,
            "store_map": False,
            "need_lane_localization": True,
            "vehicle_config": dict(
                lidar=dict(num_lasers=120, distance=50),
                lane_line_detector=dict(num_lasers=0, distance=50),
                side_detector=dict(num_lasers=12, distance=50),
                show_dest_mark=True,
                navigation_module=TrajectoryNavigation,
            ),
            "max_lateral_dist": 4
        })
        return config

    def __init__(self, config, scenario):
        super().__init__(config)
        self.scenario = scenario

    def done_function(self, vehicle_id: str):
        return False, {}

    def cost_function(self, vehicle_id: str):
        return 0, {}
    
    def reward_function(self, vehicle_id: str):
        return 0, {}

    def _get_observations(self):
        return {DEFAULT_AGENT: self.get_single_observation()}
    
    def setup_engine(self):
        self.engine.register_manager("agent_manager", self.agent_manager)
        self.engine.register_manager("map_manager", ScenarioMapManager())
        self.engine.register_manager("data_manager", InMemoryScenarioDataManager(self.scenario))


In [64]:
env = ReplayEnv(config={"use_render": True, "manual_control": False}, scenario=ScenarioDescription(data))

In [65]:
env.reset()
for i in range(1000):
    env.step([0, 1])

Known pipe types:
  glxGraphicsPipe
(1 aux display modules not yet loaded.)
:task(warning): Creating implicit AsyncTaskChain default for AsyncTaskManager TaskManager
show_dest_mark and show_line_to_dest are not supported in TrajectoryNavigation


SystemExit: 

/home/fidgetsinner/venvs/metadrive/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [66]:
env.close()